In [0]:
%scala
#fill in directory and key
spark.sparkContext.hadoopConfiguration.set()

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Row, Column
from pyspark.sql.types import ArrayType, StringType, StructType, StructField, DoubleType, TimestampType, FloatType, IntegerType

# Combine event dataframes:

From CRA:
- dfAntibioticEvents


From Slaughter:
- dfBirthEvents
- dfThinningEvents
- dfHatchEvents
- dfDeathEvents
- dfRelocationEvents

In [0]:
dfAntibioticEventsStruct = spark.read.parquet("//dfAntibioticEventsStruct.parquet")

In [0]:
dfHatchEventsStruct = spark.read.parquet("//dfHatchEventsStruct.parquet")

In [0]:
dfSlaughterEventsStruct = spark.read.parquet("//dfSlaughterEventsStruct.parquet")

In [0]:
dfThinningEventsStruct = spark.read.parquet("//dfThinningEventsStruct.parquet")

In [0]:
dfDeathEventsStruct = spark.read.parquet("//dfDeathEventsStruct.parquet")

In [0]:
dfRelocationEventsStruct = spark.read.parquet("//dfRelocationEventsStruct.parquet")

In [0]:
dfEvents2 = dfAntibioticEventsStruct \
    .unionByName(dfHatchEventsStruct, allowMissingColumns=True) \
    .unionByName(dfThinningEventsStruct, allowMissingColumns=True) \
    .unionByName(dfSlaughterEventsStruct, allowMissingColumns=True) \
    .unionByName(dfDeathEventsStruct, allowMissingColumns=True) \
    .unionByName(dfRelocationEventsStruct, allowMissingColumns=True) \
    .groupby('FarmIdentification','Pen') \
    .agg(F.sort_array(F.collect_list(F.struct("EventDate","EventType", "MetaData"))).alias("Events"))

In [0]:
dfEvents2.write \
    .format("parquet") \
    .mode("overwrite") \
    .save("//dfEvents2.parquet")

# first version: 15/09/23 (based on dfEvents with NDV data)
# edits:
# 19/09/23: changed slaughter data, now with link to PMP
# 19/09/23: changed slaughter data: now with postal code, fixed mistakes with differing hatch dates
# 24/10/23: changed antibiotic events: added subscription data